## Bayesian estimation of enzyme elasticities for an in vitro pathway

This notebook is derived from Wu2004.ipynb. 
Wu2004.ipynb uses a model and data from C. Giersch, European Journal of Biochemistry. 227, 194–201 (1995). 
However, instead of running just one model, this notebook will run analysis for a batch of models. 


In [ ]:
# handy-dandy
import os
import sys
import re
from tqdm import tqdm
import warnings
# warnings.filterwarnings("error")
# warnings.resetwarnings()
warnings.filterwarnings('ignore')

# arrays/dataframes
import numpy as np
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from csv import writer

# math/stats
import scipy
import scipy.stats
import pymc as pm
import aesara
import arviz as az

# biochemical pathway simulators
import cobra
import tellurium as te

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})

# linlog Bayesian Metabolic Control Analysis
import emll
from emll.util import initialize_elasticity

In [ ]:
import inference_utils as bi
# ba.testImport()
FOLDER_NAME = '10sp_w-Allo/'

with open(FOLDER_NAME + "passlist.txt") as file:
    passlist = [line.rstrip() for line in file]

DATA_OMISSION_CODE = 'A'
ADVI_ITERATIONS = 60000 # 30000

passN = 1 # what position is the passNumber? e.g. 1 for "data_24_pt10.csv or 2 for "mass_action_152.xml" 

In [ ]:
# make results folders 
results_dir = './' + FOLDER_NAME + 'results/'
try: 
    os.mkdir(results_dir)
    os.mkdir(results_dir + 'convergence/')
    os.mkdir(results_dir + 'elast-hdi/')
    os.mkdir(results_dir + 'elast-plot/')
    os.mkdir(results_dir + 'FCC-hdi/')
    os.mkdir(results_dir + 'FCC-graph/')
    os.mkdir(results_dir + 'MCC-hdi/')
    os.mkdir(results_dir + 'MCC-graph/')
except:
    pass

In [ ]:
for dataPath in os.listdir(FOLDER_NAME + 'generated_data/'):
    modelNo = re.split(r'[_|.]', dataPath)[passN]
    path = f'mass_action_{modelNo}.ant'
    
    try:
        bi.run_analysis(path, dataPath, itr=ADVI_ITERATIONS, folder_name=FOLDER_NAME)        
    except: 
        with open("failed10.log", "a") as f:
            f.write(dataPath + '\n')
    

In [ ]:
path = 'mass_action_718.xml'
dataPath = 'data_718_pt10.csv'
folder_name = FOLDER_NAME
itr = ADVI_ITERATIONS

In [ ]:
exSp = [i.id for i in model.reactions if 'EX' in i.id]
target_rxn_i = model.reactions.index([i for i in exSp if y_cols[-1][1:] == i[4:]][0])


In [ ]:
r = te.loads(folder_name + 'sbml/' + path)
model = cobra.io.read_sbml_model(folder_name + 'sbml/' + path)

# return r, model
data = pd.read_csv(folder_name + 'generated_data/' + dataPath).astype(float)

N = r.getFullStoichiometryMatrix()
nm, nr = N.shape

n_exp = len(data)   

e_cols = [col for col in data.columns if 'E' in col]
x_cols = r.getFloatingSpeciesIds()    
y_cols = [col for col in data.columns if 'B' in col]
v_cols = [col for col in data.columns if 'flux' in col]

e = data[e_cols]
x = data[x_cols]
y = data[y_cols]
v = data[v_cols]

# the reference index is the strain that produces the most of a desired product
# here, we arbitrarily choose the random 
ref_ind = data.idxmax()[y_cols[-1]] ## corresponds to how the data was generated
exSp = [i.id for i in model.reactions if 'EX' in i.id]
target_rxn_i = model.reactions.index([i for i in exSp if y_cols[-1][1:] == i[4:]][0])

e_star = e.iloc[ref_ind].values
x_star = x.iloc[ref_ind].values
y_star = y.iloc[ref_ind].values
v_star = v.iloc[ref_ind].values

v_star[v_star == 0] = 1e-9
y_star[y_star == 0] = 1e-6

# Normalize to reference values (and drop trivial measurement)
en = e.values / e_star
xn = x.values / x_star
yn = y.values / y_star
vn = v.values / v_star


N[:, v_star < 0] = -1 * N[:, v_star < 0]
v_star = np.abs(v_star)

Ex = emll.create_elasticity_matrix(model)
Ey = np.zeros((nr, len(y_cols))) # (reactions, number of external species)

# external species reaction indices
exSp = [model.reactions.index(i.id) for i in model.reactions if 'EX' in i.id]
for i in range(len(exSp)): 
    Ey[int(exSp[i]), i] = 1

ll = emll.LinLogLeastNorm(N, Ex, Ey, v_star)

### creating the probability model
with pm.Model() as pymc_model:
    
    # Initialize elasticities
    Ex_t = pm.Deterministic('Ex', initialize_elasticity(N, 'ex', b=0.05, sd=1, alpha=5))
    Ey_t = pm.Deterministic('Ey', initialize_elasticity(-Ey.T, 'ey', b=0.05, sd=1, alpha=5))

with pymc_model:
        
    # Error priors. 
    v_err = pm.HalfNormal('v_error', sigma=0.05, initval=.1)
    x_err = pm.HalfNormal('x_error', sigma=0.05, initval=.1)
    y_err = pm.HalfNormal('y_error', sigma=0.05, initval=.01)
    e_err = pm.HalfNormal('e_error', sigma=10, initval=.01)

    # Calculate steady-state concentrations and fluxes from elasticities
    chi_ss, vn_ss_x = ll.steady_state_aesara(Ex_t, Ey_t, en, yn)
    y_ss, vn_ss_y = ll.steady_state_aesara(Ey_t, Ex_t, en, xn)

    # Error distributions for observed steady-state concentrations and fluxes
    
    v_hat_obs = pm.Normal('v_hat_obs', mu=vn_ss_y, sigma=v_err, observed=vn) # both bn and v_hat_ss are (28,6)
    chi_obs = pm.Normal('chi_obs', mu=chi_ss,  sigma=x_err,  observed=xn) # chi_ss and xn is (28,4)
    y_obs = pm.Normal('y_obs', mu=y_ss,  sigma=y_err, observed=yn)
    e_obs = pm.Normal('e_obs', mu=0,  sigma=e_err, observed=en)

    trace_prior = pm.sample_prior_predictive() 
        # sampling
with pymc_model:
    approx = pm.ADVI()
    hist = approx.fit(n=itr, obj_optimizer=pm.adagrad_window(learning_rate=5E-3), obj_n_mc=1)
        
with pymc_model:
    trace = hist.sample(1000)    
    ppc_vi = pm.sample_posterior_predictive(trace, random_seed=1)

# label
m_labels = [m.id for m in model.metabolites]
r_labels = [r.id for r in model.reactions]
y_labels = y_cols

ex_labels = np.array([['$\epsilon_{' + '{0},{1}'.format(rlabel, mlabel) + '}$'
                    for mlabel in m_labels] for rlabel in r_labels]).flatten()
ey_labels = np.array([['$\epsilon_{' + '{0},{1}'.format(rlabel, mlabel) + '}$'
                    for mlabel in y_labels] for rlabel in r_labels]).flatten()

e_labels = np.hstack((ex_labels, ey_labels))

# generating and storing results
results_dir = './' + folder_name + 'results/'
dataset_name = dataPath.split(".")[0]

mcc_df = ADVI_CCs_hdi(trace, trace_prior, 'mcc', r, model, ll, results_dir + f'MCC-hdi/{dataset_name}-MCC_hdi.csv')


In [ ]:
path = 'mass_action_18.xml'
dataPath = 'data_18_pt10.csv'
folder_name = FOLDER_NAME
itr = ADVI_ITERATIONS

In [ ]:
r = te.loads(folder_name + 'sbml/' + path)
model = cobra.io.read_sbml_model(folder_name + 'sbml/' + path)

ex_list = [i.id for i in list(model.reactions) if 'EX' in i.id]
model.remove_reactions(ex_list)

removable_mets = [i for i in list(model.metabolites) if 'B' in i.id]
# if removable_mets in a reaction, write down the name of the reaction
# this code assumes that there will only be one source and one sink
ex_sp_rxns = [list(model.metabolites.get_by_id(m.id).reactions)[0].id for m in removable_mets]
for met in removable_mets:
    model.remove_metabolites(met)

# return r, model
data = pd.read_csv(folder_name + 'generated_data/' + dataPath).astype(float)

N = r.getFullStoichiometryMatrix()
nm, nr = N.shape

n_exp = len(data)   

e_cols = [col for col in data.columns if 'E' in col]
x_cols = r.getFloatingSpeciesIds()    
y_cols = [col for col in r.getBoundarySpeciesIds()]
v_cols = [col for col in data.columns if 'flux' in col]

e = data[e_cols]
x = data[x_cols]
y = data[y_cols]
v = data[v_cols]

ref_ind = data.idxmax()[y_cols[-1]] ## corresponds to how the data was generated

e_star = e.iloc[ref_ind].values
x_star = x.iloc[ref_ind].values
y_star = y.iloc[ref_ind].values
v_star = v.iloc[ref_ind].values

v_star[v_star == 0] = 1e-9
y_star[y_star == 0] = 1e-6

# Normalize to reference values (and drop trivial measurement)
en = e.values / e_star
xn = x.values / x_star
yn = y.values / y_star
vn = v.values / v_star[ref_ind]

N[:, v_star < 0] = -1 * N[:, v_star < 0]
v_star = np.abs(v_star)

Ex = emll.create_elasticity_matrix(model)

Ey = np.zeros((nr, len(y_cols))) # (reactions, number of external species)

# external species reaction number--the reaction number where the external species appears? (list)
ex_sp_rxn_ns = [i[1:] for i in ex_sp_rxns]

# for each external species:    
for i in range(len(removable_mets)): 
    Ey[int(ex_sp_rxn_ns[i]), i] = 1

ll = emll.LinLogLeastNorm(N, Ex, Ey, v_star)

### creating the probability model
with pm.Model() as pymc_model:
    
    # Initialize elasticities
    Ex_t = pm.Deterministic('Ex', initialize_elasticity(N, 'ex', b=0.05, sd=1, alpha=5))
    Ey_t = pm.Deterministic('Ey', initialize_elasticity(-Ey.T, 'ey', b=0.05, sd=1, alpha=5))
        
with pymc_model:
    
    # Error priors. 
    v_err = pm.HalfNormal('v_error', sigma=0.05, initval=.1)
    x_err = pm.HalfNormal('x_error', sigma=0.05, initval=.1)
    y_err = pm.HalfNormal('y_error', sigma=0.05, initval=.01)
    e_err = pm.HalfNormal('e_error', sigma=0.05, initval=.01)

    # Calculate steady-state concentrations and fluxes from elasticities
    chi_ss, v_hat_ss = ll.steady_state_aesara(Ex_t, Ey_t, en, yn)

    # Error distributions for observed steady-state concentrations and fluxes
    
    v_hat_obs = pm.Normal('v_hat_obs', mu=v_hat_ss, sigma=v_err, observed=vn) # both bn and v_hat_ss are (28,6)
    chi_obs = pm.Normal('chi_obs', mu=chi_ss,  sigma=x_err,  observed=xn) # chi_ss and xn is (28,4)
    y_obs = pm.Normal('y_obs', mu=yn,  sigma=y_err)
    e_obs = pm.Normal('e_obs', mu=en,  sigma=e_err)

    trace_prior = pm.sample_prior_predictive() 

# sampling
with pymc_model:
    approx = pm.ADVI()
    hist = approx.fit(n=itr, obj_optimizer=pm.adagrad_window(learning_rate=5E-3), obj_n_mc=1)

with pymc_model:
    trace = hist.sample(1000)    
    ppc_vi = pm.sample_posterior_predictive(trace, random_seed=1)

In [ ]:
for reaction in model.reactions:
    for metabolite, stoich in reaction.metabolites.items():
            print(metabolite.id)

In [ ]:
r.getBoundarySpeciesIds()


In [ ]:
for i in model.metabolites:
    if i.id in r.getBoundarySpeciesIds():
        i.compartment = 'e'
    else:
        i.compartment = 'c'

[i.compartment for i in model.metabolites]

In [ ]:
model = cobra.io.read_sbml_model(folder_name + 'sbml/' + path)
model.exchanges

In [ ]:
cobra.util.create_stoichiometric_matrix(model).shape

In [ ]:
model.reactions.EX_S0

In [ ]:
N